In [1]:
import os
import pandas as pd
from pathlib import Path
from fredapi import Fred
import sys
sys.path.append('../')
from src.data import fred_data

We set FED key

In [2]:
fred = Fred(os.getenv("API_FED"))
fred

It is time to create a dict of the vars and their codes to be downloaded by the FED API

In [3]:
codes_fed = {
    "crude_oil_wti" : "DCOILWTICO",
    "crude_oil_brent" : "DCOILBRENTEU",
    "gas_henry_hub" : "DHHNGSP",
    "nominal_broad_usdollar" : "DTWEXBGS",
    "yuan_dollar_spot_tc" : "DEXCHUS",
    "usdollar_euro" : "DEXUSEU",
    "treasury3month" : "DTB3",
    "treasury1year" : "DTB1YR",
    "treasury4week" : "DTB4WK",
    "treasury6month" : "DTB6"
}

We do the call

In [4]:
data_fred = {}
for k, v in codes_fed.items():
    data_fred[k] = fred.get_series(v)

In [5]:
data_fred["crude_oil_wti"]

1986-01-02    25.56
1986-01-03    26.00
1986-01-06    26.53
1986-01-07    25.85
1986-01-08    25.87
              ...  
2022-09-06    87.35
2022-09-07    82.50
2022-09-08    84.04
2022-09-09    87.27
2022-09-12    88.18
Length: 9573, dtype: float64

For creating the correct data frame we generate a fun that the logic is the same in the next code:

In [6]:
temp = data_fred["crude_oil_wti"].to_frame().reset_index()
temp = (temp.rename(columns={temp.columns[0] : "date",
                    temp.columns[1] : "value"})
            .assign(variable = "crude_oil_wti")
        )
temp

,date,value,variable
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
9568,2022-09-06,87.35,crude_oil_wti
9569,2022-09-07,82.50,crude_oil_wti
9570,2022-09-08,84.04,crude_oil_wti
9571,2022-09-09,87.27,crude_oil_wti


Here, we apply the fun

In [7]:
data_fred_clean = {}
for k, v in codes_fed.items():
    data_fred_clean[k] = fred_data.cleaning_fred(ds=data_fred[k], var_name=k)

In [8]:
data_fred_clean['gas_henry_hub']

,date,value,var
0,1997-01-07,3.82,gas_henry_hub
1,1997-01-08,3.80,gas_henry_hub
2,1997-01-09,3.61,gas_henry_hub
3,1997-01-10,3.92,gas_henry_hub
4,1997-01-13,4.00,gas_henry_hub
...,...,...,...
6701,2022-09-07,8.12,gas_henry_hub
6702,2022-09-08,8.27,gas_henry_hub
6703,2022-09-09,8.31,gas_henry_hub
6704,2022-09-12,8.20,gas_henry_hub


Concat the dataframes

In [9]:
raw_fred = pd.concat(data_fred_clean, axis=0, ignore_index=True)
raw_fred

,date,value,var
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
103448,2022-09-07,3.34,treasury6month
103449,2022-09-08,3.37,treasury6month
103450,2022-09-09,3.44,treasury6month
103451,2022-09-12,3.48,treasury6month


Saving the dataframe

In [10]:
fred_path = Path.cwd().parent.joinpath("data", "raw_fred.csv")
print(f"output path is:{fred_path}")
raw_fred.to_csv(fred_path, index=False)

output path is:c:\Documentos\proyectos\xm4ecast\data\raw_fred.csv
